# Prev

## include

In [124]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

sum_force!

## model

In [125]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 1000
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

Par1, Par2 = (0.95e-1,2.0,4.5), 0.95e-1;
# Par1, Par2 = (1e-2,2.0,3.8), 1e-2;
type = "two"
value=15

  0.000000 seconds (4 allocations: 144 bytes)


15

## review parameters

In [126]:
# Almost
# Par1, Par2 = (9e-2,2.1,5.0), 1e-1;
# Par1, Par2 = (5e-3,2.1,5.0), 5e-2;

## aggregate struct

In [127]:
if type == "one"
    agg = nothing
    # @time @start_agg FusionAGG = FusionAggregate(
    @time agg = Aggregate(
        [
            AggType(
                "HEK_1", 
                InteractionPar(
                    Cubic(Par1...), 
                    ContractilePar(Par2)
                ),
                # Float32.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
                # Float32.(readdlm("init.xyz")[3:end,2:end]) |> cu
                Float32.(readdlm("../../data/init/Sphere/$(value).0.xyz")[3:end,2:end]) |> cu
            )
        ], 
        [
            AggLocation("HEK_1",[0 0 0]),
        ],
        model
    )
elseif type == "two"
    agg = nothing
    # @time @start_agg FusionAGG = FusionAggregate(
    @time agg = FusionAggregate(
        [
            AggType(
                "HEK_1", 
                InteractionPar(
                    Cubic(Par1...), 
                    ContractilePar(Par2)
                ),
                Float32.(readdlm("init.xyz")[3:end,2:end]) |> cu
                # Float32.(readdlm("../../data/init/Sphere/$(value).0.xyz")[3:end,2:end]) |> cu
                # Float32.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu

            )
        ], 
        model
    )
end

  0.913694 seconds (1.96 M allocations: 463.841 MiB, 3.86% gc time, 0.88% compilation time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.095, 2.0, 4.5), ContractilePar(0.095)), 15.806249f0, Float32[-4.0206532 -3.3273787 -19.216244; -4.555479 -5.0831895 -18.712723; … ; -0.6608513 0.4537038 8.2605505; 2.7400308 -0.20740333 8.037631], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-19.826902 -3.3273787 -19.216244; -20.361729 -5.0831895 -18.712723; … ; 15.145397 0.4537038 8.2605505; 18.54628 -0.20740333 8.037631], AggGeometry(Float32[15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249  …  15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 15.806249, 1

## PreSet

In [128]:
points = agg.Position
# points = points[collect(size(points,1):-1:1),:]

r_max  = agg.Simulation.Parameter.Force.rₘₐₓ
idx    = agg.Simulation.Neighbor.idx_red
idx_cont = agg.Simulation.Neighbor.idx_cont
idx_sum = agg.Simulation.Neighbor.idx_sum
dist   = zeros(size(idx)) |> cu

force    = agg.Simulation.Force.F
ParForce = agg.Simulation.Parameter.Force
ParCont  = agg.Simulation.Parameter.Contractile.fₚ
outline  = agg.Geometry.outline
random = similar(force) |> cu

tₛᵢₘ  = model.Time.tₛᵢₘ
nₛₐᵥₑ = model.Time.nₛₐᵥₑ
nₖₙₙ  = model.Time.nₖₙₙ
dt = model.Time.dt

# Calculating position of every cell on the fusion
check_data("Test.xyz")
check_data("ErrorInit.xyz")

In [129]:
test = similar(idx_sum) |> cu

# Calculating Distance Matrix
threads=(64)
CUDA.@time @cuda(
    threads=threads,
    blocks=cld.(size(points,1),threads),
    dist_kernel!(idx,idx_cont,idx_sum,dist,points,r_max)
)

display(idx_sum)
display(idx)
display(dist)

  0.392377 seconds (434.33 k CPU allocations: 23.555 MiB)


1×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 9  9  9  10  14  12  8  12  6  10  8  …  13  9  13  12  9  8  10  9  11  9

1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 43   45   39  121   44    4  44  219  …  5004  5003  4796  4972  5003  4966
  2   33  134    6  141   43   5  106     4966  4949  4961  4973  4997  4889
  7  362   19  140   43   84   1  115     5007  4999  4867  5002  4866  5004
 45    1   45   51    7   37  13  122     4866  4966  4612  4958  4865  4772
 44   32   62   30   69  121  70  125     4997  5008  4974  4887  4966  4877
 37  107    2   42   70    7  53   38  …  4949  4877  4898  4889  4958  4974
  3   39    1   84  143  141   6  211     4877  4878  4967  5007  5002  4967
 33    3   38    0   84    5  43   39     4991  4998  5000  4966  5006  4999
  6   37   44   49    6   13   0  212     4998     0  5008  4772  4996  5005
  0    0    0   10   12   51   0  107        0     0  4999     0  4877     0
  0    0    0    9   17   10   0  362  …     0     0     0     0  4636     0
  0    0    0    0   53    1   0  101        0     0     0     0     0     0
  0    0    0    0   13 

1000×5008 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.90252  1.68068  1.7887   1.48638  …  1.37247  1.79264  1.81016  1.72631
 1.90327  1.73303  1.80451  1.65271     1.8939   1.87473  1.81887  1.99
 1.96492  1.87105  1.9021   1.91355     2.07097  1.93586  1.85127  1.99404
 1.98599  1.90327  1.95129  1.9329      2.07361  2.11837  1.85632  2.01755
 2.00407  1.95543  2.14129  1.95212     2.29475  2.16033  1.9603   2.07062
 2.11025  2.08931  2.17032  2.10316  …  2.3765   2.25133  2.11626  2.12342
 2.18809  2.093    2.18809  2.20381     2.43745  2.25446  2.11672  2.14111
 2.26638  2.17032  2.21044  0.0         2.55638  2.26517  2.25446  2.41218
 2.43911  2.69069  2.48545  2.24073     2.56074  2.69782  2.31013  2.56074
 0.0      0.0      0.0      2.26829     2.60758  0.0      2.5043   0.0
 0.0      0.0      0.0      2.36242  …  0.0      0.0      2.51087  0.0
 0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
 0.0      0.0      0.0      0.0         0.0      0.0      0.0

In [130]:
# t_knn = 20
# random = Int.(zeros(size(idx_sum))) |> cu
# function sumo_force!(idx,idx_cont,idx_sum,points,force,force_par,cont_par,dt,t_knn)
#     # Defining Index for kernel
#     i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
#     k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

#     # Limiting data inside matrix
#     if i <= size(points, 1) && k <= size(points, 2)

#         if idx_cont[t_knn,i] != i && idx_cont[t_knn,i] != 0
#             dist = euclidean(points,i,idx_cont[t_knn,i])
#             # force[i,k] += cont_par[i]*(points[i,k]-points[idx_cont[t_knn,i],k])/dist
#             force[i,k] = cont_par[i]*(points[i,k]-points[idx_cont[t_knn,i],k])/dist
#         end

#     end
#     return nothing
# end

# global prev_position = copy(points)
# # Compute the forces between each pair of particles in `agg` and their displacement.
# threads=(16,3)
# @cuda(
#     threads=threads,
#     blocks=(cld.(size(points,1)+1,threads[1]),1),
#     sumo_force!(idx,idx_cont,idx_sum,points,force,ParForce,ParCont,dt,t_knn)
# )

# display(random)
# display(force)

In [131]:
euclidean(points,1,2)

1.9032719f0

In [132]:
output = Int.(zeros(size(idx_sum))) |> cu

function TESTO!(idx,idx_sum,output)    
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    if i <= size(idx_sum,2)
        random = idx[rand(1:idx_sum[i]),i]
        output[i] = random
    end
    return nothing

end

threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(idx_sum,2),threads),
    TESTO!(idx,idx_sum,output)  
)

display(output)

1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 44  32  45  140  7  84  13  219  140  …  5007  5008  5008  4958  4997  5004

In [133]:
# Par = (1e-2,1.0,2.0)

# A = collect(0:0.1:2.1) |> cu
# out = zeros(size(A)) |> cu
# ForPar  = LennardJones([repeat([Par[i]],size(A,1)) for i=1:size(Par,1)]...) |> cu

# function TESTO!(A,out,ForPar)
#     i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

#     if i <= size(A,1)
#         out[i] = force_func(ForPar,i,A[i])
#     end
#     return nothing
# end

# # Calculating Distance Matrix
# threads=(64)
# CUDA.@time @cuda(
#     threads=threads,
#     blocks=cld.(size(A,1),threads),
#     TESTO!(A,out,ForPar)
# )
# display(out)


# Run

## mini test

In [134]:
# J , I = 10 , 5
# Neigbor = true

# check_data("miniTest.xyz")
# open("miniTest.xyz", "a") do f
#     write(f, "$(size(points, 1))\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(outline,Matrix(points)), ' ')
# end

# for j =1:J
#     for i = 1:I
#         # Compute the forces between each pair of particles in `agg` and their displacement.
#         threads=(16,3)
#         @cuda(
#             threads=threads,
#             blocks=(cld.(size(points,1)+1,threads[1]),1),
#             sum_force!(idx,random,idx_sum,points,force,ParForce,ParCont,dt,1)
#         )

#     end

#     if Neigbor == true
#         # Calculating Distance Matrix
#         threads=(64)
#         @cuda(
#             threads=threads,
#             blocks=cld.(size(points,1),threads),
#             dist_kernel!(idx,idx_cont,idx_sum,dist,points,r_max)
#         )
#     end

#     if j%200 == 1
#         prev_points = copy(points)
#         open("miniTest.xyz", "a") do f
#             write(f, "$(size(points, 1))\n")
#             write(f, "t=$(j*J)\n")
#             writedlm(f,hcat(outline,Matrix(points)), ' ')
#         end
#     end
# end

# val = 50
# println("BIG Val        = $(val)")
# println("Any BIG value? = $(any(points .> val))")
# println("Any Nan Value? = $(any(isnan.(points)))")
# display(points)
# display(force)

## test

In [135]:
function run_test()
    @showprogress "Forces..."  for step::Int=0:tₛᵢₘ/dt

        # Saving data in a given time (nₛₐᵥₑ)
        if (step % trunc(Int,tₛᵢₘ/nₛₐᵥₑ/dt)) == 1
            open("Test.xyz", "a") do f
                write(f, "$(size(points, 1))\n")
                write(f, "t=$(step*dt)\n")
                writedlm(f,hcat(outline,Matrix(points)), ' ')
            end
        end

        # Calculating the kNN for the aggregate
        t_nₖₙₙ = step % nₖₙₙ+1
        if t_nₖₙₙ == 1
            
            # Calculating Distance Matrix
            threads=(100)
            @cuda(
                threads=threads,
                blocks=cld.(size(points,1),threads),
                dist_kernel!(idx,idx_cont,idx_sum,dist,points,r_max)
            )

        end

        global prev_position = copy(points)
        # Compute the forces between each pair of particles in `agg` and their displacement.
        threads=(16,3)
        @cuda(
            threads=threads,
            blocks=(cld.(size(points,1)+1,threads[1]),1),
            sum_force!(idx,idx_cont,idx_sum,points,force,ParForce,ParCont,dt,t_nₖₙₙ)
        )

        # <------------------------------------------ THIS
        if any(isnan.(points)) == true 
        # || any(points .> 1e4) == true
            println("ERROR t = $(step*dt)")
            println("NaN?    = $(any(isnan.(points)))")
            println("Big?    = $(any(points .> 1e4))")
            open("ErrorInit.xyz", "a") do f
                write(f, "$(Int(size(prev_position, 1)/2))\n")
                write(f, "t=$(step * dt)\n")
                writedlm(f,hcat(outline[1:Int(size(points,1)/2),:],Matrix(points[1:Int(size(points,1)/2),:])), ' ')
            end
            break
        end
        # <------------------------------------------ THIS

    end 

end

run_test()

display(idx_sum)
display(idx)
display(findmax(idx_sum))

Forces... 100%|██████████████████████████████████████████| Time: 0:03:55


1×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 14  13  9  8  13  8  10  7  11  14  …  10  11  13  7  14  13  13  7  12  8

1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  62   43   32   37  304   70   33  …  4941  4940  5000  5007  4958  5005
  38   62  219   33   58   53   43     4455  4991  4960  4966  4966  5000
   2    1  123   31   10   69    2     4772  4997  4961  4972  4996  4966
  44    7  242   13   15   37   37        0  4998  4761  4889  5006  4801
  43   33   46   30  121    5  123     4866  4951  4796  4958  5002  5003
 263   32   39  121   70  132   32  …  4897  4949  4967  5002  4865  4889
  52  123    2    7   84   15    4     4877  4844  5008  4635  4866  4999
  33   44   19   49   13   13   31     5000  4999  4959     0  4957  4877
   0   38    7    0    9    0  132     4999  4866  5003     0  4972     0
  45   47    0    0   69    0    3     5005  4938  4877     0  4997     0
   0   19    0    0   37    0    0  …  5008  4992     0     0  4955     0
  84    3    0    0   18    0    0     4958  4878  4612     0  4948     0
 130  219    0    0    6    0    0     4966     0  4801     

(17, CartesianIndex(1, 585))